In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from solver import Solver
import os

In [ ]:
results = {}
for dir_name, _, filenames in os.walk("problems"):
    for filename in filenames:
        if filename == "problem_from_article.lp" or filename == "problem_from_article_mod.lp":
            continue
        results[filename] = [None, None, None, None]
        path = os.path.join(dir_name, filename)

        print(f"{filename} started solving with params: presolve - disable, cutting - disable.")
        sl = Solver(path, False, 0, True)
        _ = sl.start()
        mip_state = sl.result()
        results[filename][0] = mip_state.number_of_relaxations

        print(f"{filename} started solving with params: presolve - enable, cutting - disable.")
        sl = Solver(path, True, 0, True)
        _ = sl.start()
        mip_state = sl.result()
        results[filename][1] = mip_state.number_of_relaxations

        print(f"{filename} started solving with params: presolve - enable, cutting - standard.")
        sl = Solver(path, True, 1, True)
        _ = sl.start()
        mip_state = sl.result()
        results[filename][2] = mip_state.number_of_relaxations

        print(f"{filename} started solving with params: presolve - enable, cutting - yolo.")
        sl = Solver(path, True, 2, True)
        _ = sl.start()
        mip_state = sl.result()
        results[filename][3] = mip_state.number_of_relaxations

mod008inf.mps started solving with params: presolve - disable, cutting - disable.
mod008inf.mps started solving with params: presolve - enable, cutting - disable.
mod008inf.mps started solving with params: presolve - enable, cutting - standard.


In [2]:
def node_label(node) -> str:
    if node.bound.upper - node.bound.lower <= 1e-9:
        return f"{node.var.name} = {node.bound.upper}\nI={node.iteration} D={node.depth}"
    else:
        return f"{node.bound.lower}≤{node.var.name}≤{node.bound.upper}\nI={node.iteration} D={node.depth}"

def node_color(idx, node, origins):
    #print(node.var, node.iteration, node.bound)
    if idx in origins:
        return "green"
    elif node.bound.upper - node.bound.lower <= 1e-9:
        return "blue"
    elif node.var.is_general:
        return "yellow"
    return "magenta"

In [4]:
def get_pos(nodes):
    graph = {}
    for node in nodes.values():
        if (node.depth, node.iteration) not in graph:
            graph[(node.depth, node.iteration)] = [node]
        else:
            graph[(node.depth, node.iteration)].append(node)

    gr = sorted(graph.items())

    current_x = 0
    current_y = 0
    x_step, y_step = 1, -1
    current_depth = 0
    res = {}
    for (depth, it), gr_nodes in gr:
        if depth == 0 and it == 0:
            current_y = -1
        if depth != current_depth:
            current_y = 0
        for node in gr_nodes:
            res[node_label(node)] = (current_x, current_y)
            current_x += x_step
        current_y += y_step
        current_depth = depth

    return res

In [ ]:
number_of_graph = 214
nodes, edges, origins = graphes[number_of_graph][0].to_plot_info()
# fuips = graphes[number_of_graph][0].find_FUIP()
# for fuip in fuips:
#     print(node_label(graphes[number_of_graph][0].nodes[fuip]))

# for depth in graphes[number_of_graph][0].drains:
#     print(depth)
#     for node_idx in depth:
#         print(node_label(graphes[number_of_graph][0].nodes[node_idx]))
#     print()

G = nx.DiGraph()

for edge in edges:
    G.add_edge(node_label(nodes[edge[0]]), node_label(nodes[edge[1]]))

# for node_idx in origins:
#     G.add_node(node_label(nodes[node_idx]))

# pos = nx.nx_agraph.graphviz_layout(G, prog="dot")
pos = get_pos(nodes)

plt.figure(3,figsize=(150,15)) 
nx.draw(G,
        pos,
        with_labels=True, 
        font_size=8, 
        node_size=2600, 
        node_color=[node_color(idx, node, origins) for idx, node in nodes.items()]
       )

B293(4) = 1
I=0 D=10
B249(4) = 0
I=0 D=9
B221(4) = 0
I=0 D=8
B218(4) = 0
I=0 D=7
B228(4) = 0
I=0 D=6
B94(3) = 1
I=0 D=5
B43(3) = 0
I=0 D=4
B69(3) = 0
I=0 D=3
B92(4) = 0
I=0 D=2
B83(4) = 1
I=0 D=1


NameError: name 'get_pos' is not defined